In [31]:
import pandas as pd
import pymongo
import time
import datetime
import magic
import os
from concurrent.futures import ProcessPoolExecutor

In [141]:
# initialize path directories
raw_data_directory = '../raw_data/'
path_to_attachments = '../attachments/'

# paths '../attachments/years/ObjectId/files

# connect to mongodb
try:
    conn = pymongo.MongoClient()
    name = 'polisci_test12Nov16'
    db = conn[name]
    folios = db.folios
    responses = db.responses
except:
    print "Failed Connection"

In [10]:
# query 
cursor = db['folios'].find({'ARCHIVORESPUESTA': {'$regex': 'respuesta='} }).limit(500)
# load to pandas dataframe
df = pd.DataFrame(list(cursor))

In [11]:
df.head()

,ARCHIVOADJUNTOSOLICITUD,ARCHIVORESPUESTA,DEPENDENCIA,DESCRIPCIONSOLICITUD,ESTATUS,FECHARESPUESTA,FECHASOLICITUD,FOLIO,LOCALIDAD,MEDIOENTRADA,OTROSDATOS,PDFOCR,RESPUESTA,SECTOR,TIPOSOLICITUD,_id,path_to_file,year
0,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DE MEDIO AMBIENTE Y RECURSOS NATURALES,Existen rellenos sanitarios para desechos hosp...,Electrónica,2004-04-02,2004-06-01 12:25:00,'1,"{u'CODIGOPOSTAL': u'6760', u'PAIS': u'México',...",Electrónica,NaN,0,Entrega de información en medio electrónico,Medio Ambiente y Recursos Naturales,Información Pública,5827a87d833f861344e7abe9,None,2004
1,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SERVICIO DE ADMINISTRACIÓN TRIBUTARIA,CONTENIDO O DATOS PRINCIPALES DE ACUERDO 325-S...,Electrónica,2004-03-15,2004-06-01 13:23:00,'10,"{u'CODIGOPOSTAL': u'55890', u'PAIS': u'México'...",Electrónica,ESTOS DATOS SON PUBLICADOS EN LA PAGINA INTERN...,0,Entrega de información en medio electrónico,Hacienda y Crédito Público,Información Pública,5827a87d833f861344e7abea,None,2004
2,https://www.infomex.org.mx/gobiernofederal/mod...,https://www.infomex.org.mx/gobiernofederal/mod...,INSTITUTO MEXICANO DE LA PROPIEDAD INDUSTRIAL,INSTITUTO MEXICANO DE LA PROPIEDAD INDUSTRIAL ...,Electrónica,2004-01-13,2004-07-01 15:11:00,'100,"{u'CODIGOPOSTAL': u'1210', u'PAIS': u'México',...",Electrónica,NaN,0,Entrega de información en medio electrónico,Economía,Información Pública,5827a87d833f861344e7abeb,../attachments/2004/5827a87d833f861344e7abeb/a...,2004
3,NaN,https://www.infomex.org.mx/gobiernofederal/mod...,SECRETARÍA DEL TRABAJO Y PREVISIÓN SOCIAL,Número de personas laborando en la subsecretar...,Electrónica,2004-02-18,2004-01-21 12:48:00,'1000,"{u'CODIGOPOSTAL': u'6800', u'PAIS': u'México',...",Electrónica,NaN,0,Entrega de información en medio electrónico,Trabajo y Previsión Social,Información Pública,5827a87d833f861344e7abec,None,2004
4,https://www.infomex.org.mx/gobiernofederal/mod...,https://www.infomex.org.mx/gobiernofederal/mod...,INSTITUTO MEXICANO DEL SEGURO SOCIAL,SOLICITO LA DOCUMENTACION COMPROBATORIA DE LOS...,Electrónica,2004-05-20,2004-04-21 12:58:00,'10000,"{u'CODIGOPOSTAL': u'0', u'PAIS': u'México', u'...",Electrónica,EL IMSS DELEGACION COAHUILA A TRAVES DEL ARCHI...,0,Entrega de información en medio electrónico,Aportaciones a Seguridad Social,Información Pública,5827a87d833f861344e7abed,../attachments/2004/5827a87d833f861344e7abed/a...,2004


In [51]:
for row in df.iloc[0:5].iterrows():
    print row[1]['path_to_file'] == None

True
True
False
True
False


In [56]:
for row in df.iloc[0:5].iterrows():
    path_to_file = row[1]['path_to_file']
    if path_to_file == None:
        year = str(row[1]['year'])
        object_id = str(row[1]['_id'])
        path_year = path_to_attachments+year+'/'
        path_object_id = path_year+object_id+'/'
        print year, object_id, path_year,path_object_id
    else:
        path_to_file,_ = os.path.split(str(row[1]['path_to_file']))
        print path_to_file

2004 5827a87d833f861344e7abe9 ../attachments/2004/ ../attachments/2004/5827a87d833f861344e7abe9/
2004 5827a87d833f861344e7abea ../attachments/2004/ ../attachments/2004/5827a87d833f861344e7abea/
../attachments/2004/5827a87d833f861344e7abeb
2004 5827a87d833f861344e7abec ../attachments/2004/ ../attachments/2004/5827a87d833f861344e7abec/
../attachments/2004/5827a87d833f861344e7abed


In [70]:
# we determine if there exists a path to file.
# if there is a path to file, great out put that path
# if there isn't, make that path and return it
def validate_path(folio):
        path_to_file = folio['path_to_file']

        if path_to_file == None:
            # initialize variables
            year = str(folio['year'])
            object_id = str(folio['_id'])
            path_year = path_to_attachments+year+'/'

            # check path to the year
            try:
                os.stat(path_year)
            except:
                os.mkdir(path_year)

            # check path to the object_id
            path_to_file = path_year+object_id+'/'
            try:
                os.stat(path_to_file)
            except:
                os.mkdir(path_to_file)
            
            return path_to_file
        
        else:
            path_to_file,_ = os.path.split(str(row[1]['path_to_file']))
            path_to_file = path_to_file+'/'
            return path_to_file
        

In [73]:
for row in df.iloc[0:5].iterrows():
    folio = row[1]
    path_to_file = validate_path(folio)

../attachments/2004/5827a87d833f861344e7abe9/
../attachments/2004/5827a87d833f861344e7abea/
../attachments/2004/5827a87d833f861344e7abeb/
../attachments/2004/5827a87d833f861344e7abec/
../attachments/2004/5827a87d833f861344e7abed/


## Now that we have validated_the_path for the download, its time to download and save

In [112]:
import datetime
import requests
import magic
"""Determine the file_extension of a magicFile"""
def what_is_file_extension(magicFile):
    import re
    r = magic.from_file(magicFile)
    if re.search("Microsoft Word",r):
        return ".docx"
    if re.search("Composite Document File V2 Document",r):
        return ".doc"
    if re.search("PDF document",r):
        return ".pdf"
    if re.search("Rich Text Format data",r):
        return ".rtf"
    if re.search("HTML",r):
        return ".html"
    if re.search("Zip",r):
        return ".zip"
    if re.search("ASCII text",r):
        return ".txt"
    else:
        return None
    
def download_file(url,path_to_attachments):
    a_request = requests.get(url, verify = False)
    response_code = a_request.status_code
    if response_code == 200:
        
        file_name = url.split('/')[-1]
        with open(str(path_to_attachments)+'/'+file_name,'wb') as f:
            for chunk in a_request.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        
        # get file extension 
        file_extension = what_is_file_extension(str(path_to_attachments)+'/'+file_name)
        # add timestamp to file name
        time_stamp = datetime.datetime.now()
        ts = time_stamp.strftime("%d-%m-%Y")
        path_to_download = str(path_to_attachments)+'response'+'_date_'+str(ts)+str(file_extension)
        
        # rename old filename to new filename, with date tag and file extension included
        os.rename(str(path_to_attachments)+'/'+file_name,
                  str(path_to_attachments)+'response'+'_date_'+str(ts)+str(file_extension))
    else:
        time_stamp = datetime.datetime.now()
        path_to_download = None
    
    return response_code,path_to_download,time_stamp

In [102]:
for row in df.iloc[0:5].iterrows():
    folio = row[1]
    path_to_file = validate_path(folio)
    url = row[1]['ARCHIVORESPUESTA']
    response_code,path_to_download,time_stamp = download_file(url,path_to_file)
    print response_code
    print path_to_download

/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


200
../attachments/2004/5827a87d833f861344e7abe9/response_date_13-11-2016.docx


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


200
../attachments/2004/5827a87d833f861344e7abea/response_date_13-11-2016.docx


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


200
../attachments/2004/5827a87d833f861344e7abeb/response_date_13-11-2016.docx


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


200
../attachments/2004/5827a87d833f861344e7abec/response_date_13-11-2016.pdf


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


200
../attachments/2004/5827a87d833f861344e7abed/response_date_13-11-2016.doc


In [117]:
from bson.objectid import ObjectId
import datetime
def build_initial_request(folio,path_to_download,time_stamp,response_code,mongo_response):
    new_dict = {}
    
    new_dict["Folio_id"] = ObjectId(str(folio['_id']))
    new_dict["path_to_file"] = str(path_to_download)
    new_dict["response"] = [{'time':time_stamp,'response_code':response_code}]
    new_dict["Last_Modified"] = time_stamp
    
    insert = mongo_response.insert_one(new_dict)
    return insert

In [118]:
for row in df.iloc[0:5].iterrows():
    folio = row[1]
    path_to_file = validate_path(folio)
    url = row[1]['ARCHIVORESPUESTA']
    response_code,path_to_download,time_stamp = download_file(url,path_to_file)
    insert = build_initial_request(folio,path_to_download,time_stamp,response_code,responses)
    print insert

/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{'path_to_file': '../attachments/2004/5827a87d833f861344e7abe9/response_date_13-11-2016.docx', 'Folio_id': ObjectId('5827a87d833f861344e7abe9'), 'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 32, 773341), 'response': [{'response_code': 200, 'time': datetime.datetime(2016, 11, 13, 2, 9, 32, 773341)}], '_id': ObjectId('5827cb5c833f8614629d16f7')}


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{'path_to_file': '../attachments/2004/5827a87d833f861344e7abea/response_date_13-11-2016.docx', 'Folio_id': ObjectId('5827a87d833f861344e7abea'), 'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 36, 347882), 'response': [{'response_code': 200, 'time': datetime.datetime(2016, 11, 13, 2, 9, 36, 347882)}], '_id': ObjectId('5827cb60833f8614629d16f8')}


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{'path_to_file': '../attachments/2004/5827a87d833f861344e7abeb/response_date_13-11-2016.docx', 'Folio_id': ObjectId('5827a87d833f861344e7abeb'), 'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 36, 971029), 'response': [{'response_code': 200, 'time': datetime.datetime(2016, 11, 13, 2, 9, 36, 971029)}], '_id': ObjectId('5827cb60833f8614629d16f9')}


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{'path_to_file': '../attachments/2004/5827a87d833f861344e7abec/response_date_13-11-2016.pdf', 'Folio_id': ObjectId('5827a87d833f861344e7abec'), 'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 37, 458061), 'response': [{'response_code': 200, 'time': datetime.datetime(2016, 11, 13, 2, 9, 37, 458061)}], '_id': ObjectId('5827cb61833f8614629d16fa')}


/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/rsampana/mex-project/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:838: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/security.html
  InsecureRequestWarning)


{'path_to_file': '../attachments/2004/5827a87d833f861344e7abed/response_date_13-11-2016.doc', 'Folio_id': ObjectId('5827a87d833f861344e7abed'), 'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 38, 96618), 'response': [{'response_code': 200, 'time': datetime.datetime(2016, 11, 13, 2, 9, 38, 96618)}], '_id': ObjectId('5827cb62833f8614629d16fb')}


# Now what if we ping again and we want to add the new ping to the response collection

In [160]:
for row in df.iloc[0:5].iterrows():
    object_id = row[1]['_id']
    r = responses.find_one({'Folio_id': object_id})
    if r == None:
        print "none"# Make the new response entry
    else:
        print r

{u'path_to_file': u'../attachments/2004/5827a87d833f861344e7abe9/response_date_13-11-2016.docx', u'Folio_id': ObjectId('5827a87d833f861344e7abe9'), u'_id': ObjectId('5827cb5c833f8614629d16f7'), u'response': [{u'response_code': 200, u'time': datetime.datetime(2016, 11, 13, 2, 9, 32, 773000)}], u'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 32, 773000)}
{u'path_to_file': u'../attachments/2004/5827a87d833f861344e7abea/response_date_13-11-2016.docx', u'Folio_id': ObjectId('5827a87d833f861344e7abea'), u'_id': ObjectId('5827cb60833f8614629d16f8'), u'response': [{u'response_code': 200, u'time': datetime.datetime(2016, 11, 13, 2, 9, 36, 347000)}], u'Last_Modified': datetime.datetime(2016, 11, 13, 2, 9, 36, 347000)}
{u'path_to_file': u'../attachments/2004/5827a87d833f861344e7abeb/response_date_13-11-2016.docx', u'Folio_id': ObjectId('5827a87d833f861344e7abeb'), u'_id': ObjectId('5827cb60833f8614629d16f9'), u'response': [{u'response_code': 200, u'time': datetime.datetime(2016, 11, 13, 2

In [ ]:
%%writefile response_inserts.py

# initialize path directories
raw_data_directory = '../raw_data/'
path_to_attachments = '../attachments/'

# paths '../attachments/years/ObjectId/files

# connect to mongodb
try:
    conn = pymongo.MongoClient()
    name = 'polisci_test12Nov16'
    db = conn[name]
    folios = db.folios
    responses = db.responses
except:
    print "Failed Connection"

# query to find the urls in ARCHIVORESPUESTA
cursor = db['folios'].find({'ARCHIVORESPUESTA': {'$regex': 'respuesta='} })

# load to pandas dataframe
df = pd.DataFrame(list(cursor))

for row in df.iloc[0:5].iterrows():
    folio = row[1]
    # get object_id
    object_id = folio['_id']
    
    # validate the path
    path_to_file = validate_path(folio)

    # get the url from 'ARCHIVORESPUESTA'
    url = folio['ARCHIVORESPUESTA']

    # download the file from the url
    response_code,path_to_download,time_stamp = download_file(url,path_to_file)
    
    # find one entry of Folio_id containing object_id
    r = responses.find_one({'Folio_id': object_id})

    # we determine if response entry already exists
    if r == None:
        # insert the file to mongodb
        insert = build_initial_request(folio,path_to_download,time_stamp,response_code,responses)
    else:
        # build the update
        update = build_update(folio,path_to_download,time_stamp,response_code,responses)

## Note To Ryan: Remember after a certain number of reponse_codes/time --> this will fill up the 16MB dedicated for entry. We need to handle for this eventuallity

In [127]:
fr['response']

0    [{u'response_code': 200, u'time': 2016-11-13 0...
Name: response, dtype: object

In [129]:
x = {'response_code':200, 'time': datetime.datetime.now()}

y = {'response_code':200, 'time': datetime.datetime.now()}

z = {'response_code':200, 'time': datetime.datetime.now()}

In [135]:
responses = [x,y,z]
print responses[1]['response_code']

200


In [ ]:
""" Read from Mongo and Store into Pandas DataFrame """
def read_mongo(db, collection, query={}, nb_limit=None, host='localhost', port=27017,
               username=None, password=None, no_id=True):

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    
    if nb_limit == None:
        # just query
        cursor = db[collection].find(query)
    else:
        cursor = db[collection].find(query).limit(nb_limit)
    
    df = pd.DataFrame(list(cursor))
    
    if no_id :
        del df['_id']
    return df